Mount the drive

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Download the relevant files

In [3]:
! gdown --id 17V5zE73X6uySSS7-ay7lrgh5VOZkJR0Y # test images
! gdown --id 1lVVphSGxfo-tJRgf13Dh9c3lEJzYFD1s # train images
! gdown --id 1IRTxGio5SNfiIGXIfMD91kZRL7i6CVSM # train labels

/usr/local/lib/python3.9/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=17V5zE73X6uySSS7-ay7lrgh5VOZkJR0Y
To: /content/test_images.npy
100% 62.7M/62.7M [00:00<00:00, 137MB/s]
/usr/local/lib/python3.9/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1lVVphSGxfo-tJRgf13Dh9c3lEJzYFD1s
To: /content/train_images.npy
100% 157M/157M [00:01<00:00, 116MB/s]
/usr/local/lib/python3.9/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/

Set up the fixed variables, functions and classes

In [40]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch.cuda
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

batch_size = 10
train_images = np.load("train_images.npy")
train_labels = pd.read_csv("train_labels.csv")['label']
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epochs = 15
label_str = {0: 'T-shirt/Top', 1: 'Trouser', 2: 'Pullover', 3: 'Dress', 4: 'Coat', 5: 'Sandal', 6: 'Shirt',
             7: 'Sneaker', 8: 'Bag', 9: 'Ankle Boot'}
test_images = np.load("test_images.npy")

def show_image(arr):
    two_d = (np.reshape(arr, (28, 28)) * 255).astype(np.uint8)
    plt.imshow(two_d, interpolation='nearest')
    plt.show()

class MNIST(Dataset):
    def __init__(self, data, labels):
        self.x = data
        self.y = labels
        self.n_samples = len(data)

    def __len__(self):
        return self.n_samples

    def __getitem__(self, index):
        return torch.from_numpy(self.x[index]).unsqueeze(0).float() / 255, self.y[index]


class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32), nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2))
        
        self.fc1 = nn.Linear(in_features=64*6*6, out_features=600)
        self.drop = nn.Dropout2d(0.25)
        self.fc2 = nn.Linear(in_features=600, out_features=120)
        self.fc3 = nn.Linear(in_features=120, out_features=10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.drop(out)
        out = self.fc2(out)
        out = self.fc3(out)
        
        return out

Set up the model and its hyperparameters

In [41]:
training_dataset = MNIST(train_images, train_labels)
dataloader = DataLoader(dataset=training_dataset, batch_size=batch_size, shuffle=True)
model = CNN()
model.to(device)
learning_rate = 0.0005

model.train()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

Train the model

In [42]:
for epoch in range(1, epochs+1):
    t_loss = 0
    t_correct = 0
    accuracies = []

    for image, label in dataloader:
        image = image.to(device)
        label = label.to(device)

        prediction = model(image)
        loss = criterion(prediction, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        t_loss += loss.item()
        t_correct += prediction.argmax(dim=1).eq(label).sum().item()
        accuracy = t_correct * 100 / len(training_dataset)
        print("Epoch no.", epoch, "|accuracy: ", round(accuracy, 3), "%", "|total_loss: ", t_loss)

/usr/local/lib/python3.9/dist-packages/torch/nn/functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Streaming output truncated to the last 5000 lines.
Epoch no. 15 |accuracy:  0.02 % |total_loss:  0.06820086389780045
Epoch no. 15 |accuracy:  0.038 % |total_loss:  0.29717301577329636
Epoch no. 15 |accuracy:  0.056 % |total_loss:  0.4548647925257683
Epoch no. 15 |accuracy:  0.074 % |total_loss:  0.6094824597239494
Epoch no. 15 |accuracy:  0.094 % |total_loss:  0.6932595670223236
Epoch no. 15 |accuracy:  0.112 % |total_loss:  0.8037374168634415
Epoch no. 15 |accuracy:  0.13 % |total_loss:  1.0170786529779434
Epoch no. 15 |accuracy:  0.15 % |total_loss:  1.0434050373733044
Epoch no. 15 |accuracy:  0.168 % |total_loss:  1.2851071916520596
Epoch no. 15 |accuracy:  0.188 % |total_loss:  1.3131582494825125
Epoch no. 15 |accuracy:  0.204 % |total_loss:  1.691774046048522
Epoch no. 15 |accuracy:  0.224 % |total_loss:  1.759008077904582
Epoch no. 15 |accuracy:  0.244 % |total_loss:  1.771407756023109
Epoch no. 15 |accuracy:  0.264 % |total_loss:  1.8193775741383433
Epoch no. 15 |accuracy:  0.28

Save the model

In [43]:
torch.save(model.state_dict(), r'/content/drive/MyDrive/Assignment 4/Weights5.pth')

Test the model over 'n' tries

In [44]:
model.eval()
predictions = []
num_test = 10
start = 100
with torch.no_grad():
  for test in range(len(test_images)):
      img = torch.tensor(test_images[test]).float() / 255
      img = img.unsqueeze(0).unsqueeze(0)

      confidence = model(img).tolist()[0]
      prediction = confidence.index(max(confidence))
      predictions.append(prediction)

      # print('Predicted Label:', label_str[int(prediction)])
      # show_image(test_images[test])
      # print('\n')

Save the prediction results of the testing set

In [45]:
import csv

with open(r'/content/drive/MyDrive/Assignment 4/submission4.csv', 'w', newline='') as f:
  writer = csv.writer(f)
  writer.writerow(['ID', 'label'])
  for i in range(len(predictions)):
    writer.writerow([i, predictions[i]])

The cell below is for testing purposes

In [46]:
print(len(predictions))

20000
